In [66]:
import os 
import matplotlib.pyplot as plt
from pathlib import Path 
from numpy import zeros
from numpy import asarray
import numpy as np
import cv2
import pydicom
import xml.etree.ElementTree as ET
from lxml import etree
from keras import applications

In [67]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras.applications.vgg16 import VGG16
from keras.layers import Input, Flatten, Dense
from keras.engine import  Model

In [51]:
#class LungDataset(Dataset):
    def extract_boxes(self, filepath):
        tree = ET.parse(filepath)
        # get the root of the document
        root = tree.getroot()
        # extract each bounding box
        boxes = list()
        for box in root.findall('.//bndbox'):
            xmin = int(box.find('xmin').text)
            ymin = int(box.find('ymin').text)
            xmax = int(box.find('xmax').text)
            ymax = int(box.find('ymax').text)
            coors = [xmin, ymin, xmax, ymax]
            boxes.append(coors)
        # extract image dimensions
        width = int(root.find('.//size/width').text)
        height = int(root.find('.//size/height').text)
        return boxes, width, height
    
    def save_dataset(self, anno_dict, dicom_dict):
        self.add_class("dataset", 1, "lungs")
        for key, value in anno_dict.items():
            if key in dicom_dict:
                self.add_image('dataset', image_id=key, path=dicom_dict.get(key), annotation=value)
    
    def load_mask(self, image_id):
        info = self.image_info[image_id]
        path = info['annotation']
        boxes, w, h = self.extract_boxes(path)
        masks = zeros([h, w, len(boxes)], dtype='uint8')
        class_ids = list()
        for i in range(len(boxes)):
            box = boxes[i]
            row_s, row_e = box[1], box[3]
            col_s, col_e = box[0], box[2]
            masks[row_s:row_e, col_s:col_e, i] = 1
            class_ids.append(self.class_names.index('lungs'))
        return masks, asarray(class_ids, dtype='int32')
    
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']
    

In [3]:
def get_anno_dict():
    annotation = os.walk('/Users/samanthasullivan/Desktop/Prac_II_Code/saved_files/other data/Annotation', topdown=False)
    anno = []
    anno_dict = {}
    for root, dir, files in annotation:
        for file in files:
            if file.endswith(".xml"):
                anno_full_path = os.path.join(root, file)
                anno_id = file.replace('.xml', '')
                anno_dict[anno_id] = anno_full_path
    return anno_dict

In [9]:
#Get dicom images
def get_dicom_dict(anno_dict, limit=0):
    dicomFiles = os.walk('/Users/samanthasullivan/Desktop/Prac_II_Code/saved_files/manifest-1608669183333/Lung-PET-CT-Dx', topdown=False)
    dcm_dict = {}
    for root, dir, files in dicomFiles:
        for file in files:
            if file.endswith(".dcm"):
                fullPath = os.path.join(root, file)
                dicomInfo = pydicom.read_file(fullPath, force=True)
                dicomFileIdentifier = dicomInfo.SOPInstanceUID;
                if(dicomFileIdentifier in anno_dict):
                    dcm_dict[dicomFileIdentifier] = fullPath
                    if(limit > 0 and len(dcm_dict) > limit):
                        return dcm_dict
                    if(len(dcm_dict)%1000 == 0):
                        print(len(dcm_dict))
    return dcm_dict

In [10]:
anno_dict = get_anno_dict()
dcm_dict = get_dicom_dict(anno_dict, 500)
print(len(anno_dict))
print(len(dcm_dict))

30993
501


In [38]:
#class LungConfig(Config):
 #   NAME = "lung_cfg"
  #  NUM_CLASSES= 1 +1
   # STEPS_PER_EPOCH = 1

In [69]:
length = len(anno_dict)
split_index = round(0.8*length)
anno_keys = list(anno_dict.keys())
train_annos = {}
test_annos = {}
for i in range(0, len(anno_keys)):
    key = anno_keys[i]
    val = anno_dict[key]
    if(i < split_index):
        train_annos[key] = val
        continue
    test_annos[key] = val
print(len(train_annos))
print(len(test_annos))

train_lungs = LungDataset()
train_lungs.save_dataset(train_annos, dcm_dict)
train_lungs.prepare()

test_lungs = LungDataset()
test_lungs.save_dataset(test_annos, dcm_dict)
test_lungs.prepare()


24794
6199


In [ ]:
class DataGenerator(keras.utils.Sequence):
  

  def __init__(self, list_IDs, datafiles, batch_size = 32, n_classes = 20, shuffle = True):
    self.batch_size = batch_size
    self.datafiles = datafiles
    self.list_IDs = list_IDs
    self.n_classes = n_classes
    self.shuffle = shuffle
    self.on_epoch_end()

  def on_epoch_end(self):
    self.indexes = np.arange(len(self.list_IDs))
    if self.shuffle == True:
      np.random.shuffle(self.indexes)

  def __data_generation(self, list_IDs_temp):

    # Initialization
    X = np.empty((self.batch_size, 224, 224, 3))
    y = np.empty((self.batch_size, 20))

    for datactr in range (self.batch_size):
      newlabel, newimg = generate_from_xml(self.datafiles[list_IDs_temp[datactr]])

      y[datactr] = newlabel
    
      X[datactr] = newimg

    

    return X, y

  def __len__(self):
    return int(np.floor(len(self.list_IDs) / self.batch_size))


  def __getitem__(self, index):
  # Generate indexes of the batch
    newindexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

  # Find list of IDs
    list_IDs_temp = [self.list_IDs[k] for k in newindexes]

  # Generate data
    X, y = self.__data_generation(list_IDs_temp)

    return X, y

In [60]:
model = Sequential()

model.add(Conv2D(input_shape=(224,224,3),filters=64,
kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,
kernel_size=(3,3),padding="same", activation="relu"))

model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3),
padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3),
padding="same", activation="relu"))

model.add(Flatten())

model.add(Dense(units=4096,activation="relu"))


In [61]:
model.compile(optimizer= keras.optimizers.RMSprop(lr=0.001), loss='binary_crossentropy',
metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_99 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_100 (Conv2D)          (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_101 (Conv2D)          (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_102 (Conv2D)          (None, 112, 112, 128)     147584    
_________________________________________________________________
conv2d_103 (Conv2D)          (None, 112, 112, 512)     590336    
_________________________________________________________________
flatten_4 (Flatten)          (None, 6422528)           0         
__________

In [70]:
earlyStopping = EarlyStopping(monitor='val_loss', verbose=0, mode='min', patience = 4)

history = model.fit_generator(generator=train_lungs,
validation_data=test_lungs,
use_multiprocessing=True,
workers=6,
steps_per_epoch = 20,
callbacks=[earlyStopping])

TypeError: object of type 'LungDataset' has no len()

In [71]:
earlyStopping = EarlyStopping(monitor='val_loss', verbose=0, mode='min', patience = 4)

history = model.fit(train_lungs,
validation_data=test_lungs,
use_multiprocessing=True,
workers=6,
steps_per_epoch = 20,
callbacks=[earlyStopping])

TypeError: fit() missing 1 required positional argument: 'y'

In [43]:
#model = MaskRCNN(mode='training', model_dir='./', config=config)
#model.load_weights('mask_rcnn_coco.h5', by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])
#model.train(train_lungs, test_lungs, learning_rate=config.LEARNING_RATE, epochs=1, layers='heads')


Starting at epoch 0. LR=0.001

Checkpoint Path: ./lung_cfg20210420T1136/mask_rcnn_lung_cfg_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
mrcnn_mask_bn4         (TimeDis

/opt/anaconda3/envs/py35/lib/python3.6/site-packages/skimage/transform/_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. Starting from version 0.19 a ValueError will be raised instead of this warning.
  order = _validate_interpolation_order(image.dtype, order)


Epoch 1/1


/opt/anaconda3/envs/py35/lib/python3.6/site-packages/skimage/transform/_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. Starting from version 0.19 a ValueError will be raised instead of this warning.
  order = _validate_interpolation_order(image.dtype, order)
/opt/anaconda3/envs/py35/lib/python3.6/site-packages/skimage/transform/_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. Starting from version 0.19 a ValueError will be raised instead of this warning.
  order = _validate_interpolation_order(image.dtype, order)
/opt/anaconda3/envs/py35/lib/python3.6/site-packages/skimage/transform/_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicite

/opt/anaconda3/envs/py35/lib/python3.6/site-packages/skimage/transform/_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. Starting from version 0.19 a ValueError will be raised instead of this warning.
  order = _validate_interpolation_order(image.dtype, order)
/opt/anaconda3/envs/py35/lib/python3.6/site-packages/skimage/transform/_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. Starting from version 0.19 a ValueError will be raised instead of this warning.
  order = _validate_interpolation_order(image.dtype, order)
/opt/anaconda3/envs/py35/lib/python3.6/site-packages/skimage/transform/_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicite

...

/opt/anaconda3/envs/py35/lib/python3.6/site-packages/skimage/transform/_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. Starting from version 0.19 a ValueError will be raised instead of this warning.
  order = _validate_interpolation_order(image.dtype, order)
/opt/anaconda3/envs/py35/lib/python3.6/site-packages/skimage/transform/_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. Starting from version 0.19 a ValueError will be raised instead of this warning.
  order = _validate_interpolation_order(image.dtype, order)
/opt/anaconda3/envs/py35/lib/python3.6/site-packages/skimage/transform/_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicite

/opt/anaconda3/envs/py35/lib/python3.6/site-packages/skimage/transform/_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. Starting from version 0.19 a ValueError will be raised instead of this warning.
  order = _validate_interpolation_order(image.dtype, order)
/opt/anaconda3/envs/py35/lib/python3.6/site-packages/skimage/transform/_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. Starting from version 0.19 a ValueError will be raised instead of this warning.
  order = _validate_interpolation_order(image.dtype, order)
/opt/anaconda3/envs/py35/lib/python3.6/site-packages/skimage/transform/_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicite

/opt/anaconda3/envs/py35/lib/python3.6/site-packages/skimage/transform/_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. Starting from version 0.19 a ValueError will be raised instead of this warning.
  order = _validate_interpolation_order(image.dtype, order)
/opt/anaconda3/envs/py35/lib/python3.6/site-packages/skimage/transform/_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. Starting from version 0.19 a ValueError will be raised instead of this warning.
  order = _validate_interpolation_order(image.dtype, order)
/opt/anaconda3/envs/py35/lib/python3.6/site-packages/skimage/transform/_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicite

/opt/anaconda3/envs/py35/lib/python3.6/site-packages/skimage/transform/_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. Starting from version 0.19 a ValueError will be raised instead of this warning.
  order = _validate_interpolation_order(image.dtype, order)
/opt/anaconda3/envs/py35/lib/python3.6/site-packages/skimage/transform/_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. Starting from version 0.19 a ValueError will be raised instead of this warning.
  order = _validate_interpolation_order(image.dtype, order)
/opt/anaconda3/envs/py35/lib/python3.6/site-packages/skimage/transform/_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicite

StopIteration: 